In [1]:

import orjson

from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps
import shapely
from shapely.wkb import loads as load_wkb
import random

In [2]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [3]:
from sqlalchemy_utils import database_exists, create_database
engine = create_engine('postgresql://postgres@localhost:5333/test')#,echo=True)

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    with dbapi_connection.cursor() as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS postgis;')

In [5]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

    @property
    def shapely_geom(self):
        return load_wkb(self.geom.desc) if self.geom else None

/tmp/ipykernel_15615/2057255873.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
# Create the table
Base.metadata.create_all(engine)

In [8]:
%%timeit
# -- orm approach
from sqlalchemy.orm import Session

# Getting the total number of rows
with Session(engine) as session:
    total_rows = session.query(GeometryModel).count()
print(total_rows)    

1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
107 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
from sqlalchemy import create_engine, MetaData, Table, select, func
import random
with  engine.connect() as conn:
    total_rows = conn.execute(text("select count(*) from geometries;")).fetchone()[0]
print(total_rows)


1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
1063260
104 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit

# ORM approach
random_index = random.randint(1, total_rows)

# Retrieve the random row by ID and convert the geom to a Shapely object
with Session(engine) as session:
    random_row = session.query(GeometryModel).filter_by(id=random_index).first()
    #--- approach 1 - return only id
    #print(random_row.id, end=" " )
    #--- approach 2 - inline convert to shapely
    # geom = random_row.shapely_geom #force convert
    # print(random_row.id, end = " ") #but don't print polygon because too much info
    #--- approach 3 - inline convert to shapely and compute area
    print(random_row.shapely_geom.area, end = " ") ## uses property to compute dynamically


624.0 12440.0 25.0 4976.0 525.0 52.5 726.0 867.5 1005.5 922.5 650.0 822.0 453.5 4.0 341.5 980.5 705.0 910.5 87080.0 823.5 477.0 582.5 893.5 63280.0 384.5 486.5 1155.5 5.5 74.5 2516.5 4.0 155424.0 461.5 780.5 1201.0 1006.0 75.5 455.0 91952.0 1452.0 408.5 718.0 59886.0 520.5 78.5 1148.5 485.0 975.0 19264.0 34.5 1562.5 1070.5 761.0 822.5 981.5 1265.0 36904.0 797.5 259.5 265.0 902.5 972.5 37088.0 1011.0 1175.5 563.0 695.0 1787.0 540.0 881.5 37520.0 1000.0 919.0 995.0 167.5 627.5 928.5 27536.0 711.0 1036.5 944.0 374.5 1220.5 52192.0 829.0 336.5 98216.0 426.5 278.5 766.0 77.0 597.0 22744.0 4905.0 519.5 614.5 91184.0 1561.5 810.5 758.0 970.0 332.5 441.5 417.5 621.5 856.0 1001.5 538.5 3669.5 497.0 910.0 650.5 15455.0 580.5 496.0 1062.0 887.0 720.0 875.0 274692.0 881.5 794.5 554.5 53743.0 436.0 67756.0 849.5 183.0 582.0 735.0 95992.0 55024.5 63.0 344.0 963.5 21.5 371.5 22.0 1001.0 498.0 505.0 4272.5 625.0 507.5 309.5 1012.5 38.5 4.5 583.5 1156.0 36648.0 833.5 858.0 536.5 1019.5 974.5 684.0 544.

In [14]:
%%timeit
# -- get the geom directly, and convert to shapely object via WKB
random_index = random.randint(1, total_rows)

with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id,geom
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    obj=shapely.wkb.loads(random_row[1])
#print(random_row[0],end=" ") # lenght of the goejson -- shold be > 0
print(obj.area,end=" ") # lenght of the goejson -- shold be > 0

884.0 838.5 19.0 994.5 63944.5 499.5 921.0 1092.0 806.5 53056.0 1783.0 3.0 1115.0 3.5 1076.0 85628.0 909.0 280.5 91009.5 777.0 105.0 279.5 1102.5 805.0 905.0 207760.0 536.5 378.5 828.0 1219.0 254.0 65.5 7.0 146.0 67864.0 473.5 189.5 240320.0 29.0 764.0 420.0 95576.0 1049.0 110.0 259.0 734.5 1907.0 612.0 377.0 1673.5 784.0 392.5 1060.5 730.5 68488.0 659.0 462.5 58600.0 514.5 733.5 5.0 16.0 1101.0 918.5 1029.0 1144.5 453.0 55976.0 924.5 425.0 1200.0 316.0 495.0 888.0 754.5 55.5 854.0 335.5 737.0 178.0 568.5 611.0 754.5 882.5 770.5 1017.5 65117.5 38071.5 21096.0 109560.0 16448.0 776.5 83072.0 568.5 507.0 764.5 918.0 17984.0 1121.0 716.5 50.5 25.5 898.0 29696.0 499.0 2365.5 417.5 660.0 119.0 458.5 246944.0 17509.0 400.0 10.5 417.5 564.0 77.0 866.0 894.0 712.5 234016.0 717.0 1877.0 25.5 1018.0 984.0 707.0 207592.0 793.5 550.5 93008.0 20080.0 976.0 758.0 745.5 1815.0 2111.5 992.0 426.5 745.5 405.5 299.0 345.0 952.0 802.0 1272.5 710.5 3722.5 7.0 1460.0 212.0 2049.0 914.0 1219.0 660.0 1021.5 4

In [15]:
%%timeit

# convert to geojson in the database, and return the string
random_index = random.randint(1, total_rows)

with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(geom) as geom 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
print(len(random_row[1]),end=" ") # lenght of the goejson -- shold be > 0

1548 1198 827 1030 1030 1282 22310 8730 1268 12398 1450 1254 372 907 1632 2808 1016 6882 1212 134 1520 946 1324 1898 1422 23710 1100 8870 1506 1268 19706 1142 1184 29898 1436 3312 988 1212 12020 1478 1240 1324 7456 1310 1030 1520 1058 9346 1366 918 14890 1394 1100 1212 1240 1212 904 1856 1170 1240 1324 778 1380 890 1408 1030 17004 1128 176 344 25250 1576 1142 1254 1128 1478 9150 554 1632 1324 190 1212 610 1170 526 11866 1271 1422 1394 162 1408 344 862 4866 2570 1212 1422 1226 918 1100 1030 1674 1464 803 5706 1968 1366 974 1296 1478 1590 1338 1184 1184 876 1154 288 7946 14036 3382 1296 680 778 1324 1492 1394 12146 1184 1352 1170 190 288 6994 1198 1324 1002 1380 2430 988 1226 22002 1534 470 15730 1436 1254 1492 1310 1324 1520 1380 1044 13322 1142 1422 1716 1268 1030 1366 1016 1380 1324 1422 9626 1206 218 1282 1800 11418 1296 540 1240 1114 1114 1296 134 1268 2934 302 1100 1184 19314 1296 1072 1100 1212 1170 1534 22730 16808 974 1352 1282 14946 1422 1282 8506 260 10298 946 1297 1128 1058 1

In [16]:
%%timeit
# with compute centroid dynamic
random_index = random.randint(1, total_rows)

with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(geom) as geom
        FROM geometries
        WHERE id = {random_index}
        ''')
    ).fetchone()
obj=shapely.from_geojson(random_row[1])
print(obj.area,end=" ") # lenght of the goejson -- shold be > 0

1853.5 785.0 678.5 873.0 226.5 878.0 464.0 1594.5 8.0 42552.0 1065.0 324048.0 1038.5 3021.0 282.0 15632.0 2437.0 52.0 556.0 84624.0 131480.0 599.0 2051.5 589.5 160.0 961.0 4751.5 6154.5 770.5 16.0 726.5 647.0 593.5 879.0 779.0 548.5 1060.5 634.5 377.5 625.5 733.0 571.0 47352.5 587.5 175.0 122.0 694.5 1054.0 328.0 732.5 6289.5 343.0 680.5 48.5 737.0 512.0 493.5 1107.5 833.5 30.0 1058.0 84576.0 892.5 711.0 65128.0 4573.0 697.5 416.0 666.5 188.0 1020.5 165.0 821.5 40.0 7130.0 5.0 1175.5 927.0 549.5 9960.0 399.0 11.0 1149.5 164.5 945.0 848.0 547.0 455.5 27552.0 639.5 34.0 32152.0 635.0 801.0 329.0 333.5 70.5 31936.0 891.0 881.5 527.0 745.0 328248.0 53.5 1040.5 616.0 1267.0 796.5 633.0 543.0 127729.5 727.0 39464.0 52.5 1569.5 50.5 339.5 220.5 805.5 879.0 813.5 779.5 14251.0 809.0 3323.0 682.5 79928.0 841.5 1038.5 29.0 1007.5 583.0 58264.0 746.0 1034.0 428.5 423.5 1012.0 68.5 95660.0 5.0 664.5 415.5 256284.5 921.0 734.5 1212.5 1002.5 194.0 1071.0 932.0 60.5 455.5 14.0 1018.5 749.0 25920.0 66

In [17]:
%%timeit
# with compute centroid dynamic
random_index = random.randint(1, total_rows)

with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(geom) as geom, ST_AsGeoJSON(ST_Centroid(geom)) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
print(len(random_row[1]),end=" ") # lenght of the goejson -- shold be > 0

1076 540 1044 1394 1310 998 1450 1576 708 1898 1562 1506 162 1394 1450 12398 1310 35554 778 2136 11040 1310 876 1408 2304 1464 1254 1016 10522 638 15674 10060 1254 1268 10522 1310 1170 1282 1422 29800 1170 14428 1352 1464 19650 2738 1184 855 1268 21022 1114 1618 1030 26678 1063 10914 1324 1492 1170 1282 3578 1842 1422 1366 1646 1086 1492 1492 1324 2304 1352 1310 1422 1100 1310 8870 932 9780 1296 1063 20238 1212 1002 1282 1366 1464 1338 820 372 1180 260 1296 134 1534 5538 1226 1016 1590 988 1254 1282 1128 1142 750 10634 960 1141 1450 1394 1422 190 1352 15002 1156 1828 1170 1254 1128 652 8674 1156 1282 1324 1310 316 1464 4572 1212 1548 1534 680 2080 568 29030 358 2556 904 1072 15226 1464 1366 1954 1198 708 1310 6784 1156 18922 15632 1618 1716 1282 960 36100 974 1464 1660 1688 1282 848 12706 1380 736 8506 848 1282 1394 442 1184 70386 1422 1478 12426 1170 1226 1193 1310 1086 1024 5020 918 1226 1016 14442 1296 1226 1016 1464 1464 1016 1058 1436 1128 1436 1310 374 15394 218 8590 1282 1618 87

### below are spatial query tests

In [ ]:
%%timeit
random_index = random.randint(1, total_rows)
    
half_bbox_size= 500
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(ST_Centroid(geom)) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    bounding_box_polygons = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(geom) as geom 
        FROM geometries 
        WHERE ST_Intersects(
            geom,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



print(len(bounding_box_polygons), end= " " )

In [ ]:
%%timeit
random_index = random.randint(1, total_rows)
    
half_bbox_size= 6_000
# Step 3: Query for the specific row based on the random index
with engine.connect() as conn:
    random_row = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(centroid) as centroid 
        FROM geometries 
        WHERE id = {random_index}
        ''')
    ).fetchone()
    
    centroid_x,centroid_y=orjson.loads(random_row[1])['coordinates']

    bounding_box_polygons = conn.execute(
        text(f'''
        SELECT id, ST_AsGeoJSON(geom) as geom 
        FROM geometries 
        WHERE ST_Intersects(
            centroid,
            ST_MakeEnvelope(
                {centroid_x - half_bbox_size}, {centroid_y - half_bbox_size},
                {centroid_x + half_bbox_size}, {centroid_y + half_bbox_size},
                4326
            )
        )
        ''')
     ).fetchall()



print(len(bounding_box_polygons), end= " " )